# **MDP를 모를 때 최고의 정책 찾기**

지금까지 MDP를 모를 때 각 상태의 가치를 어떻게 평가할 수 있는지 공부하였음

이제 MDP를 모를 때 최고의 정책을 찾는 방법을 공부하겠음

(Montecarlo Control, SARSA, Q-Learning)

## **1. 몬테카를로 컨트롤**

### **1) 개요**
챕터 4에서 MDP를 알 때 최적의 정책 함수를 찾는 방법론을 배웠음. 그 중 정책 이터레이션은 임의의 정책에서 시작하여 정책을 평가하고 밸류를 계산하고 그리디 정책을 만드는 과정을 계속해서 반복하는 방법이었는데, 지금 상황과의 차이는 MDP를 모른다는 점에서만 차이가 난다.


**1) 정책을 평가하기 위한 상태별 보상과 전이확률을 알 수 없다.**
$$v_\pi(s) = \sum_{a{\in}A}\pi(a|s)\biggl(r^a_s + \gamma\sum_{s'{\in}S}P^a_{ss'}v_{\pi}(s')\biggl)$$ 

$$ *여기서 전이확률과 보상을 모르는 상태 $$

 - 따라서 직접 액션을 해보기 전까지는 다음 상태가 어디일지, 얼마의 확률로 도착한 것인지, 보상은 얼마인지 알 수 없음

**2) 어떻게든 상태별 보상에 대한 평가를 마쳤더라도 정책 개선 단계에서 그리디한 정책을 만들 수 없다.**
 - 전이확률을 안다 치고 $s0$에서 $a1$과 , $a2$ 액션을 했을 때 어느 상태로 도착할 지 알 수 없기 때문에 그리디한 정책을 채택할 수 없음

### **2) 해결방법**

**1) 평가자리에 MC**

 - 각 상태의 밸류를 계산하기 위해 MC또는 TD를 활용한다.
    
**2) V 대신 Q**

 - 상태가치함수 $v(s)$만 가지고는 **정책개선**에서 그리디 정책을 생성할 수 없으니 $v(s)$대신 행동가치함수 $q(s,a)$를 활용
 - 상태 $s0$에서 액션 $a1$, $a2$ 액션을 했을 때 상태 $s1$, $s2$로 전이된 $v(s1)$, $v(s2)$를 구하는 것이 아니라 $q(s0, a1)$, $q(s0, a2)$를 화룡한다는 의미
 - 이를 활용하면 $q(s0, a1) = 1$, $q(s0, a2) = 2$를 알 수 있기 때문에 그리디 액션을 선택할 수 있음

**3) greedy 대신 decaying $\epsilon$-greedy(내가 아는 그 입실론 그리디)**
여기서 갑자기 $\epsilon$-greedy가 튀어나온 이유는 무엇인가?
 - MDP를 모르는 상황이기 때문에 MC를 이용하여 q(s,a)를 계산하고, 그리디 정책을 만들고, 다시 MC를 이용해 q(s,a)를 계산하는 과정을 반복하게 된다.
 - 문제는 그리디한 정책하에서 한 번 q(s0,a1) = 0.1로 계산하면, 다른 액션을 했을 때q(s0,a2)=0(모르니까 0임, 초기화)이기 때문에 무조건 q(s0,a1)만 선택하게 된다.
 - 따라서 $\epsilon$을 부여해서 탐색의 정도를 설정하는 것
 $$\pi(a|s) = \begin{cases}1 - \epsilon& \text{if a* = argmax_a q(s,a)}\\
            \epsilon &\text{otherwise}\end{cases}$$

    - 만약 $\epsilon$을 0.1로 가정하면, 90%확률로 $q(s,a)$가 높은 액션을 선택하고, 10%의 확률로 랜덤하게 액션을 선택한다.(탐색과 활용)
    - 단, 학습 초기에는 $\epsilon$을 높게 잡았다가 데이터가 쌓이게 되면 활용에 집중하기 위해 $\epsilon$을 낮춤

### **3) 몬테카를로 컨트롤 구현**

**MCC 요약**

 1) 간략화된 정책 이터레이션을 활용함(정책 평가와 개선을 수렴할 때까지 진행하지 않음, 챕터 4 참고)
 
 2) 한 에피소드의 경험을 쌓고
 
 3) (정책평가) 경험한 데이터로 $q(s,a)$ 테이블의 값을 업데이트하고
 
 4) (정책개선) 업데이트된 $q(s,a)$ 테이블을 이용하여 $\epsilon$-greedy 정책을 만들고
 
 5) (반복) 이를 반복한다.

In [16]:
import pandas as pd
pd.DataFrame([['길','길','벽','길','길','길','길'], 
              ['길','길','벽','길','길','길','길'],
              ['시작','길','벽','길','벽','길','길'],
              ['길','길','길','길','벽','길','길'],
             ['길','길','길','길','벽','길','종료']])

,0,1,2,3,4,5,6
0,길,길,벽,길,길,길,길
1,길,길,벽,길,길,길,길
2,시작,길,벽,길,벽,길,길
3,길,길,길,길,벽,길,길
4,길,길,길,길,벽,길,종료


In [3]:
from ch6_MCControl import GridWorld, QAgent

## GridWorld 클래스

## QAgent 클래스

In [17]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # q벨류를 저장하는 변수. 모두 0으로 초기화. (5개의 7 * 4 그리드월드)
        self.eps = 0.9 # 입실론 0.9부터 시작
        self.alpha = 0.01
        
    def select_action(self, s):
        # eps-greedy로 액션을 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, history):
        # 한 에피소드에 해당하는 history를 입력으로 받아 q 테이블의 값을 업데이트 한다
        cum_reward = 0
        for transition in history[::-1]:
            s, a, r, s_prime = transition
            x,y = s
            # 몬테 카를로 방식을 이용하여 업데이트.
            self.q_table[x,y,a] = self.q_table[x,y,a] + self.alpha * (cum_reward - self.q_table[x,y,a])
            cum_reward = cum_reward + r 

    def anneal_eps(self):
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)

    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q 값이 가장 높았는지 보여주는 함수
        q_lst = self.q_table.tolist()
        data = np.zeros((5,7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

## main함수

In [36]:
def main():
    env = GridWorld()
    agent = QAgent()

    for n_epi in range(1000): # 총 1,000 에피소드 동안 학습
        done = False
        history = []

        s = env.reset()
        while not done: # 한 에피소드가 끝날 때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            history.append((s, a, r, s_prime))
            s = s_prime
        agent.update_table(history) # 히스토리를 이용하여 에이전트를 업데이트
        agent.anneal_eps()

    agent.show_table() # 학습이 끝난 결과를 출력

if __name__ == '__main__':
    main()
print("epsilon에 의해 결괏값은 매번 달라질 수 있음(확률적 선택)")


[[3. 3. 0. 2. 2. 2. 3.]
 [3. 3. 0. 2. 2. 2. 3.]
 [2. 3. 0. 1. 0. 3. 3.]
 [1. 2. 2. 1. 0. 2. 3.]
 [3. 2. 2. 1. 0. 2. 0.]]
epsilon에 의해 결괏값은 매번 달라질 수 있음(확률적 선택)


array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [21]:
def main():
    env = GridWorld()
    agent = QAgent()

    for n_epi in range(1): # 총 1,000 에피소드 동안 학습
        done = False
        history = []

        s = env.reset()
        while not done: # 한 에피소드가 끝날 때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            history.append((s, a, r, s_prime))
            s = s_prime
            
        agent.update_table(history) # 히스토리를 이용하여 에이전트를 업데이트
        print(history)
        agent.anneal_eps()

    agent.show_table() # 학습이 끝난 결과를 출력

if __name__ == '__main__':
    main()

[((0, 0), 3, -1, (1, 0)), ((1, 0), 1, -1, (0, 0)), ((0, 0), 1, -1, (0, 0)), ((0, 0), 0, -1, (0, 0)), ((0, 0), 3, -1, (1, 0)), ((1, 0), 2, -1, (1, 1)), ((1, 1), 3, -1, (2, 1)), ((2, 1), 1, -1, (1, 1)), ((1, 1), 3, -1, (2, 1)), ((2, 1), 1, -1, (1, 1)), ((1, 1), 2, -1, (1, 1)), ((1, 1), 1, -1, (0, 1)), ((0, 1), 1, -1, (0, 1)), ((0, 1), 1, -1, (0, 1)), ((0, 1), 2, -1, (0, 1)), ((0, 1), 3, -1, (1, 1)), ((1, 1), 0, -1, (1, 0)), ((1, 0), 2, -1, (1, 1)), ((1, 1), 2, -1, (1, 1)), ((1, 1), 3, -1, (2, 1)), ((2, 1), 0, -1, (2, 0)), ((2, 0), 3, -1, (3, 0)), ((3, 0), 1, -1, (2, 0)), ((2, 0), 2, -1, (2, 1)), ((2, 1), 0, -1, (2, 0)), ((2, 0), 0, -1, (2, 0)), ((2, 0), 0, -1, (2, 0)), ((2, 0), 2, -1, (2, 1)), ((2, 1), 3, -1, (3, 1)), ((3, 1), 1, -1, (2, 1)), ((2, 1), 1, -1, (1, 1)), ((1, 1), 0, -1, (1, 0)), ((1, 0), 1, -1, (0, 0)), ((0, 0), 2, -1, (0, 1)), ((0, 1), 0, -1, (0, 0)), ((0, 0), 2, -1, (0, 1)), ((0, 1), 0, -1, (0, 0)), ((0, 0), 1, -1, (0, 0)), ((0, 0), 1, -1, (0, 0)), ((0, 0), 0, -1, (0, 0)),

In [20]:
def main():
    # env, agent 인스턴스
    env = GridWorld()
    agent = Agent()
    # 테이블 초깃값 설정
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    # 보상, 할인율, 업데이트 파라미터 정의
    gamma = 1.0
    reward = -1
    alpha = 0.001
    
    # 총 5만 번의 에피소드를 진행하는데, 
    for k in range(50000):
        done = False
        history = []
        
        # 에이전트가 경험을 쌓는 과정. not done이 True이면(done이 False인게 True면) 아래 변수들을 만드는데, done = True이면 not done은 False이니까 멈춘다. 
        while not done:
            # agent는 select_action()을 하고
            action = agent.select_action()
            # (x,y)의 좌표, 보상, 중단여부 변수를 만들고(env에서 정의한 step 함수, is_done 함수 등에 의해 (self.x, self.y), reward, done 를 리턴)
            (x,y), reward, done = env.step(action)
            # 그리고 그 값을 history에 append한다.
            history.append((x,y,reward))
        # 에피소드가 한 번 끝나면 환경 리셋
        env.reset()
        
        
        # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        
        # 쌓은 경험을 통해 테이블을 업데이트하는 영역
        # transition이 history 테이블의 역순 정렬(append 된 시점 기준 역순)에 있으면, == 방문했던 상태들을 뒤에서부터 보겠다는 의미
        # G_t와 G_t+1은 재귀적 관계가 있으므로 역순으로 정렬(G_t = R_t+1 + gammaG_t+1)
        for transition in history[::-1]:
            # x, y, reward에 transition의 밸류를 넣고,
            x, y, reward = transition
            # 테이블에서 state별 밸류를 업데이트 한다. V(s_t) <- V(s_t) + alpha(G_t - V(s_t))
            data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])
            # 각 state의 cum_reward를 업데이트 한다.
            cum_reward = cum_reward + gamma*reward
#             print(data[x][y])
            
    print("업데이트 결과: ")           
    for row in data:
        # row 출력 - 굳이 for를 쓰는 건 그대로 print(data)하면 list로 나오니깐 보기 불편함
        # 여기서 row에 있는 값은 data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])를 통해 업데이트한 값
        print(row)
        
if __name__ == '__main__':
    main()

업데이트 결과: 
[-60.790709554349945, -58.3862365554446, -53.3399901121054, -47.65292391597761]
[-56.528237590692726, -53.80719151893177, -48.02114054484271, -41.73786634594]
[-56.553299207497226, -50.86077709516569, -40.298866020675796, -28.88894104534365]
[-54.53325344736141, -45.9914139601591, -29.829851181883573, 0.0]


In [21]:
def main():
    # env, agent 인스턴스
    env = GridWorld()
    agent = Agent()
    # 테이블 초깃값 설정
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    # 보상, 할인율, 업데이트 파라미터 정의
    gamma = 1.0
    reward = -1
    alpha = 0.001
    
    # 총 5만 번의 에피소드를 진행하는데, 
    for k in range(3):
        done = False
        history = []
        
        # 에이전트가 경험을 쌓는 과정. not done이 True이면(done이 False인게 True면) 아래 변수들을 만드는데, done = True이면 not done은 False이니까 멈춘다. 
        while not done:
            # agent는 select_action()을 하고
            action = agent.select_action()
            # (x,y)의 좌표, 보상, 중단여부 변수를 만들고(env에서 정의한 step 함수, is_done 함수 등에 의해 (self.x, self.y), reward, done 를 리턴)
            (x,y), reward, done = env.step(action)
            # 그리고 그 값을 history에 append한다.
            history.append((x,y,reward))
        print("history: " + str(history))
        # 에피소드가 한 번 끝나면 환경 리셋
        env.reset()
        
        
        # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        
        # 쌓은 경험을 통해 테이블을 업데이트하는 영역
        # transition이 history 테이블의 역순 정렬(append 된 시점 기준 역순)에 있으면, == 방문했던 상태들을 뒤에서부터 보겠다는 의미
        # G_t와 G_t+1은 재귀적 관계가 있으므로 역순으로 정렬(G_t = R_t+1 + gammaG_t+1)

        for transition in history[::-1]:
            # x, y, reward에 transition의 밸류를 넣고,
            x, y, reward = transition
            # 테이블에서 state별 밸류를 업데이트 한다. V(s_t) <- V(s_t) + alpha(G_t - V(s_t))
            data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])
            # 각 state의 cum_reward를 업데이트 한다.
            cum_reward = cum_reward + gamma*reward
            print("state({}, {})의 누적 보상: ".format(x, y) +str(cum_reward))
            
        print("\n")
            
    print("업데이트 결과: ")           
    for row in data:
        # row 출력 - 굳이 for를 쓰는 건 그대로 print(data)하면 list로 나오니깐 보기 불편함
        # 여기서 row에 있는 값은 data[x][y] = data[x][y] + alpha*(cum_reward-data[x][y])를 통해 업데이트한 값
        print(row)

if __name__ == '__main__':
    main()

history: [(1, 0, -1), (1, 1, -1), (2, 1, -1), (1, 1, -1), (2, 1, -1), (2, 2, -1), (3, 2, -1), (3, 3, -1)]
state(3, 3)의 누적 보상: -1.0
state(3, 2)의 누적 보상: -2.0
state(2, 2)의 누적 보상: -3.0
state(2, 1)의 누적 보상: -4.0
state(1, 1)의 누적 보상: -5.0
state(2, 1)의 누적 보상: -6.0
state(1, 1)의 누적 보상: -7.0
state(1, 0)의 누적 보상: -8.0


history: [(1, 0, -1), (0, 0, -1), (0, 1, -1), (0, 0, -1), (0, 0, -1), (0, 1, -1), (1, 1, -1), (2, 1, -1), (1, 1, -1), (2, 1, -1), (3, 1, -1), (3, 2, -1), (3, 1, -1), (3, 1, -1), (3, 0, -1), (3, 1, -1), (3, 2, -1), (3, 2, -1), (3, 2, -1), (2, 2, -1), (1, 2, -1), (2, 2, -1), (1, 2, -1), (0, 2, -1), (0, 1, -1), (0, 0, -1), (0, 0, -1), (0, 0, -1), (1, 0, -1), (1, 0, -1), (1, 1, -1), (2, 1, -1), (2, 2, -1), (3, 2, -1), (3, 3, -1)]
state(3, 3)의 누적 보상: -1.0
state(3, 2)의 누적 보상: -2.0
state(2, 2)의 누적 보상: -3.0
state(2, 1)의 누적 보상: -4.0
state(1, 1)의 누적 보상: -5.0
state(1, 0)의 누적 보상: -6.0
state(1, 0)의 누적 보상: -7.0
state(0, 0)의 누적 보상: -8.0
state(0, 0)의 누적 보상: -9.0
state(0, 0)의 누적 보상: -10.0
state(0, 1)

In [66]:
t = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]
# t[1][1]
for row in t:
    print(row)

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 16]


In [15]:
t = [1, 2, 3, 4, 5]
t[::-1]

[5, 4, 3, 2, 1]